In [1]:
with open('libraries.py') as f:
    code = f.read()
exec(code)

with open('functions.py') as f:
    code = f.read()
exec(code)

In [2]:
pd.reset_option('all')
pd.set_option('display.max_rows', 100)

In [3]:
# determine user
user = getpass.getuser()
if user == 'peymansh':
    main_folder_path = '/Users/peymansh/Dropbox (MIT)/Research/AI and Occupations/ai-exposure'
    data_path = f'{main_folder_path}/output'

In [4]:
onet_data_path = f'{data_path}/data/onet_occupations_yearly.csv'

# list of occupations to create DAGs for
occupation_list = ['travelAgents', 'insuranceUnderwriters', 'pileDriverOperators', 
                   'dredgeOperators', 'gradersAndSortersForAgriculturalProducts', 'reinforcingIronAndRebarWorkers',
                   'insuranceAppraisersForAutoDamage', 'floorSandersAndFinishers', 'dataEntryKeyer', 
                   'athletesAndSportsCompetitors', 'audiovisualEquipmentInstallerAndRepairers', 'hearingAidSpecialists', 
                   'personalCareAides', 'proofreadersAndCopyMarkers', 'chiropractors', 
                   'shippingReceivingAndInventoryClerks', 'cooksShortOrder', 'orthodontists',
                   'subwayAndStreetcarOperators', 'packersAndPackagersHand', 'hoistAndWinchOperators', 
                   'forgingMachineSettersOperatorsAndTenders', 'avionicsTechnicians', 'dishwashers', 
                   'dispatchersExceptPoliceFireAndAmbulance', 'familyMedicinePhysicians', 'MachineFeedersAndOffbearers'
                   ]

occupation = 'travelAgents'
occupation = 'insuranceUnderwriters'
occupation = 'pileDriverOperators'

# Generate occupation-specific strings
GPT_input_occupation, plot_title_occupation, occupation_code, occupation_folder = pick_occupation(occupation)

In [5]:
# set alpha as AI quality metric
n = 100
epsilon = 1e-8
alpha_list = np.linspace(epsilon, 1-epsilon, n).tolist()

### Initialize input-output paths

In [6]:
# Manual DAG
input_path = f'{occupation_folder}/{occupation}_M_DAG_df.csv'
output_path = f'{occupation_folder}/{occupation}_costMin_M.csv'

# First Last Task DAG
input_path = f'{occupation_folder}/{occupation}_FLT_GPT_DAG_df.csv'
output_path = f'{occupation_folder}/{occupation}_costMin_FLT.csv'

# Conditioned First Last Task DAG
input_path = f'{occupation_folder}/{occupation}_CFLT_GPT_DAG_df.csv'
output_path = f'{occupation_folder}/{occupation}_costMin_CFLT.csv'

# Partitioned DAG
input_path = f'{occupation_folder}/{occupation}_P_GPT_DAG_df.csv'
output_path = f'{occupation_folder}/{occupation}_costMin_P.csv'

# Conditioned Partitioned DAG
input_path = f'{occupation_folder}/{occupation}_CP_GPT_DAG_df.csv'
output_path = f'{occupation_folder}/{occupation}_costMin_CP.csv'

In [7]:
# read DAG
dag_df = pd.read_csv(input_path)

# remove edges if comment column labeled with "TriangleRemovedFlag" (edge is there for plotting purposes and is not part of the actual DAG)
if 'comment' in dag_df.columns:
    dag_df = dag_df[~dag_df['comment'].str.endswith('TriangleRemovedFlag')]

# get task stats
tasks_stats = pd.read_csv(f'{occupation_folder}/{occupation}_taskStats.csv')
tasks_stats

# print stats
#tasks_stats.iloc[:,1:].sum()
dag_df

,source,target,comment
0,Move hand and foot levers of hoisting equipmen...,Drive pilings to provide support for buildings...,The worker driving the pilings needs to know t...
2,Drive pilings to provide support for buildings...,Move levers and turn valves to activate power ...,The worker operating the levers and valves to ...
3,Conduct pre-operational checks on equipment to...,"Clean, lubricate, and refill equipment.","The worker responsible for cleaning, lubricati..."
4,Conduct pre-operational checks on equipment to...,Move hand and foot levers of hoisting equipmen...,The worker moving hand and foot levers to posi...
5,"Clean, lubricate, and refill equipment.",Move hand and foot levers of hoisting equipmen...,The worker operating the hoisting equipment ne...
6,Conduct pre-operational checks on equipment to...,Drive pilings to provide support for buildings...,The worker driving pilings needs to know that ...
8,Conduct pre-operational checks on equipment to...,Move levers and turn valves to activate power ...,The worker who is moving levers and turning va...
10,"Clean, lubricate, and refill equipment.","""Target""",Job Completion Indicator
11,Move levers and turn valves to activate power ...,"""Target""",Job Completion Indicator


In [8]:
# extract list of tasks and create a dictionary for indexing tasks
tasks_list = tasks_stats['task'].unique()
tasks_dict = {i: node for i, node in enumerate(tasks_list, start=0)}

# create numpy array of adjacency matrix
adjacency_matrix = np.zeros((len(tasks_list), len(tasks_list)), dtype=int)
aux_dict = {value: key for key, value in tasks_dict.items()}
for _, row in dag_df.iterrows():
    source_index = aux_dict[row['source']]
    target_index = aux_dict[row['target']]
    adjacency_matrix[source_index, target_index] = 1

tasks_dict

{0: 'Move hand and foot levers of hoisting equipment to position piling leads, hoist piling into leads, and position hammers over pilings.',
 1: 'Conduct pre-operational checks on equipment to ensure proper functioning.',
 2: 'Drive pilings to provide support for buildings or other structures, using heavy equipment with a pile driver head.',
 3: 'Move levers and turn valves to activate power hammers, or to raise and lower drophammers that drive piles to required depths.',
 4: 'Clean, lubricate, and refill equipment.',
 5: '"Target"'}

In [ ]:
def find_neighbors(adjacency_matrix):
    n = adjacency_matrix.shape[0]  # Number of nodes
    neighbors_dict = {}

    for node in range(n):
        neighbors = []
        for neighbor in range(n):
            if adjacency_matrix[node, neighbor] != 0:  # Check for an edge from node to neighbor
                neighbors.append(neighbor)
        neighbors_dict[node] = neighbors

    # add node itself to set of its neighbors
    for key in neighbors_dict.keys():
        neighbors_dict[key].append(key)

    return neighbors_dict

In [ ]:
from itertools import combinations

def get_node_inclusive_partitions(arr, element):
    # Ensure the element is in the list
    if element not in arr:
        return []

    # Remove the specific element from the list
    arr.remove(element)
    
    # Generate all subsets of the remaining elements
    subsets = []
    for r in range(len(arr) + 1):
        subsets.extend(combinations(arr, r))
    
    # Add the specific element back to each subset
    subsets_with_element = []
    for subset in subsets:
        subsets_with_element.append((element,) + subset)

    subsets_with_element = [list(subset) for subset in subsets_with_element]
    return subsets_with_element

In [ ]:
def keep_unique_partitions(my_list):
    unique_values_tuple = set(tuple(sorted(inner_list)) for inner_list in my_list)
    unique_values_lists = [list(inner_tuple) for inner_tuple in unique_values_tuple]
    unique_values_lists.sort(key=len)
    return unique_values_lists

In [ ]:
def get_subsets_with_all_elements(input_list, must_include):
    # Generate all possible subsets
    all_subsets = []
    for r in range(len(input_list) + 1):
        subsets = combinations(input_list, r)
        all_subsets.extend(subsets)
    
    # Filter subsets to include only those that contain all elements from must_include except the must_include itself
    valid_subsets = [list(subset) for subset in all_subsets if all(elem in subset for elem in must_include) and list(subset) != must_include]
    
    return valid_subsets

In [ ]:
def get_valid_DAG_subsets(adjacency_matrix):
    # subset adjacency matrix to exclude Target node
    non_target_adjacency_matrix = adjacency_matrix[:-1,:-1].copy()

    # get neighbors of each task
    tasks_neighbors = find_neighbors(non_target_adjacency_matrix)

    # get list of tasks with no outgoing edges (i.e., last tasks)
    last_tasks_list = [key for key, value in tasks_neighbors.items() if len(value) == 1]

    # get number of non-Target nodes
    n = non_target_adjacency_matrix.shape[0]

    # initialize dict for valid subsets of nodes (and also partitions) to act as memory
    memory_dict = {key: [] for key in range(n)}
    

    # initialize dictionary for valid subsets origniating from each node
    valid_subsets_dict = {}

    def valid_subsets_recursive(partition):

        # if partition already in memory return its value
        try:
            if len(memory_dict[tuple(sorted(partition))]) > 0:
                return memory_dict[tuple(partition)]
            
        # if partition not in memory, get valid subsets of partition
        except KeyError:
            # get neighbors of partition
            neighbors_list = set(np.where(non_target_adjacency_matrix[partition, :].sum(axis=0) > 0)[0])
            neighbors_list = list(neighbors_list.difference(partition)) # remove partition nodes from neighbors list

            # if partition is terminal (i.e., has no outgoing edges), return partition + empty list 
            if len(neighbors_list) == 0:
                memory_dict[tuple(sorted(partition))] = [partition]
                return [partition]
            # if partition is non-terminal, run recursive function on all partitions of neighbors
            else:
                neighbors_list = set(np.where(non_target_adjacency_matrix[partition, :].sum(axis=0) > 0)[0])
                neighbors_list = list(neighbors_list.difference(partition)) # remove partition nodes from neighbors list
                # add partition nodes to neighbors list
                for node in partition:
                    neighbors_list.append(node)
                
                # get all subsets of neighbors list that contain elements of partition
                partitions_of_partition = get_subsets_with_all_elements(neighbors_list, partition)
                
                partitions_of_partition_valid_subsets = []
                for partition_of_partition in partitions_of_partition:
                    partition_of_partition_valid_subsets = valid_subsets_recursive(partition_of_partition)

                    # add partition nodes to neighbor's valid subsets
                    partition_of_partition_valid_subsets = [list(set(partition + subset)) for subset in partition_of_partition_valid_subsets]

                    # add partition itself as a valid subset
                    partition_of_partition_valid_subsets.append(partition)
                    partition_of_partition_valid_subsets = keep_unique_partitions(partition_of_partition_valid_subsets)

                    # update memory for partition of current partition
                    memory_dict[tuple(sorted(partition_of_partition))] = partition_of_partition_valid_subsets

                    partitions_of_partition_valid_subsets.extend(partition_of_partition_valid_subsets)
                    partitions_of_partition_valid_subsets = keep_unique_partitions(partitions_of_partition_valid_subsets)

                # update memory for partition
                memory_dict[tuple(sorted(partition))] = partitions_of_partition_valid_subsets            
                return partitions_of_partition_valid_subsets
            

    for node in range(n):
        valid_subsets_dict[node] = valid_subsets_recursive([node])

    # modify valid subsets of last tasks
    for last_task in last_tasks_list:
        last_task_neighbors = [i for i in range(n) if non_target_adjacency_matrix[i, last_task] != 0]
        last_task_neighbors.append(last_task)
        
        node_partitions = get_node_inclusive_partitions(last_task_neighbors, last_task)

        # update valid_subsets_dict for last task
        # exclude partitions with only two nodes as they already appear in parent node partitions
        valid_subsets_dict[last_task] = [partition for partition in node_partitions if len(partition) != 2]


    return valid_subsets_dict


In [ ]:
valid_subsets_dict = get_valid_DAG_subsets(adjacency_matrix)
valid_subsets_dict

In [ ]:
def is_combination_valid(combination, n):
    # Flatten list of combination
    covered_tasks_list = [element for sublist in combination for element in sublist]
    
    # Create a set of the flattened list
    covered_tasks_set = set(covered_tasks_list)
    
    # Check if the flattened set has exactly n elements and contains all elements from 0 to n-1
    if len(covered_tasks_list) == n and covered_tasks_set == set(range(n)):
        return True
    else:
        return False


def generate_combinations(valid_subsets_dict, current_key=0, current_combination=None, result=None):
    if current_combination is None:
        current_combination = []
    if result is None:
        result = []

    # Base case: if convered all tasks add current combination to the result list
    if is_combination_valid(current_combination, len(valid_subsets_dict)):
        result.append(current_combination)
        return result

    # Recursive case: iterate through the list of lists at the current key
    for subset in valid_subsets_dict[current_key]:
        # Create a new combination including the current subset
        new_combination = current_combination + [subset]
        new_combination_flattened = [element for sublist in new_combination for element in sublist]

        # Check which nodes are NOT covered by the new combination. Only need to process these nodes next
        uncovered_nodes = list(set(range(len(valid_subsets_dict))) - set(new_combination_flattened))

        if len(uncovered_nodes) == 0:
            if is_combination_valid(new_combination, len(valid_subsets_dict)):
                result.append(new_combination)
                return result
        else:
            # Recursively call the function to process the next key
            for nex_key in uncovered_nodes:
                generate_combinations(valid_subsets_dict, nex_key, new_combination, result)

    return result

In [ ]:
combinations = []
for initial_node in range(len(valid_subsets_dict)):
    combinations.extend(generate_combinations(valid_subsets_dict, initial_node))

In [ ]:
def normalize_sublist(sublist):
    # Sort the elements within each inner list and then sort the entire sublist
    return tuple(sorted(tuple(sorted(inner)) for inner in sublist))

def unique_lists(input_list):
    seen = set()
    unique_combinations = []

    for sublist in input_list:
        normalized = normalize_sublist(sublist)
        if normalized not in seen:
            seen.add(normalized)
            unique_combinations.append(sublist)

    return unique_combinations

In [ ]:
unique_combinations = unique_lists(combinations)
print(len(unique_combinations))